<hr><hr><br>

# Welcome to <strong>Gerald</strong>
<p>Gerald is a machine learning model that predict RL matchups based on past results. He doesn't dive into stats. He doesn't look at positioning. He doesn't look at goals, assists, boost, or any other in-game details. Gerald just looks at which players win and how different players interact on teams together and against other teams.</p><br>
<p><strong>If you want to use Gerald without looking at the coding, skip to the next <i>text (not code)</i> section for instructions. </strong></p><br>
<p>Everything before the next segments is model-training/model-building code that I have left in for those that want to look through. You can skip quickly using the table of contents</p>
<br><hr><hr>

#Model Creation Section (skip for predictions)

##Data Collection

In [1]:
%pip install torchmetrics
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.4 MB/s eta 0:00:00


In [2]:
import time
import requests
import re
import random
import pickle
import json
import torch
import math
import random
import optuna

import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn.functional as f
import torch.nn.init as init

from collections import deque, Counter
from datetime import datetime, timedelta
from scipy import stats
from torch import nn
from torchmetrics import Accuracy
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from pathlib import Path
from google.colab import userdata
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Ballchasing_Token = userdata.get("BALLCHASING_TOKEN")

In [ ]:
class RateLimiter:
    #Token bucket rate limiter with dual constraints
    def __init__(self, requests_per_second=2, requests_per_hour=1000):
        self.rps = requests_per_second
        self.rph = requests_per_hour
        self.second_window = deque()
        self.hour_window = deque()

    def wait_if_needed(self):
        now = time.time()

        # Clean old entries
        cutoff_second = now - 1.0
        cutoff_hour = now - 3600.0

        while self.second_window and self.second_window[0] < cutoff_second:
            self.second_window.popleft()
        while self.hour_window and self.hour_window[0] < cutoff_hour:
            self.hour_window.popleft()

        # Wait if at limits
        if len(self.second_window) >= self.rps:
            sleep_time = 1.0 - (now - self.second_window[0])
            if sleep_time > 0:
                time.sleep(sleep_time)

        if len(self.hour_window) >= self.rph:
            sleep_time = 3600.0 - (now - self.hour_window[0])
            if sleep_time > 0:
                print(f"Hourly limit reached. Sleeping for {sleep_time/60:.1f} minutes...")
                time.sleep(sleep_time)

        # Record this request
        now = time.time()
        self.second_window.append(now)
        self.hour_window.append(now)


class BallchasingCollector:
    #Scalable data collector with checkpointing and error handling

    def __init__(self, token, checkpoint_dir="/content/drive/MyDrive/checkpoints"):
        self.token = token
        self.headers = {
            'Authorization': token,
            'Content-Type': 'application/json'
        }
        self.base_url = "https://ballchasing.com/api"
        self.rate_limiter = RateLimiter()
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)

        # Statistics
        self.stats = {
            'requests_made': 0,
            'errors': 0,
            'replays_processed': 0,
            'start_time': None
        }

    def _make_request(self, url, max_retries=3):
        #Make rate-limited request with exponential backoff
        for attempt in range(max_retries):
            try:
                self.rate_limiter.wait_if_needed()
                response = requests.get(url, headers=self.headers, timeout=30)
                self.stats['requests_made'] += 1

                if response.status_code == 429:  # Rate limited
                    wait_time = min(2 ** attempt * 5, 60)
                    print(f"Rate limited. Waiting {wait_time}s...")
                    time.sleep(wait_time)
                    continue

                response.raise_for_status()
                return response.json()

            except requests.exceptions.RequestException as e:
                self.stats['errors'] += 1
                print(f"Error on attempt {attempt + 1}/{max_retries}: {e}")
                if attempt == max_retries - 1:
                    raise
                time.sleep(2 ** attempt)

        return None

    def discover_replay_groups(self, group_ids, checkpoint_file="replay_groups.pkl"):
        #Discover all leaf groups containing replays using BFS
        checkpoint_path = self.checkpoint_dir / checkpoint_file

        # Try to load from checkpoint
        if checkpoint_path.exists():
            print(f"Loading replay groups from checkpoint...")
            with open(checkpoint_path, 'rb') as f:
                return pickle.load(f)

        print(f"Discovering replay groups from {len(group_ids)} root groups...")
        replay_groups = []
        queue = deque(group_ids)
        processed = set()

        while queue:
            group_id = queue.popleft()

            if group_id in processed:
                continue
            processed.add(group_id)

            print(f"  Checking group: {group_id}")

            try:
                data = self._make_request(f"{self.base_url}/groups?group={group_id}")

                for sub_group in data.get('list', []):
                    if sub_group['direct_replays'] > 0:
                        replay_groups.append(sub_group['id'])
                        print(f"    ✓ Found {sub_group['direct_replays']} replays")
                    else:
                        queue.append(sub_group['id'])

            except Exception as e:
                print(f"Error processing {group_id}: {e}")
                continue

        # Save checkpoint
        with open(checkpoint_path, 'wb') as f:
            pickle.dump(replay_groups, f)

        print(f"Discovered {len(replay_groups)} groups with replays")
        return replay_groups

    def get_replay_ids(self, replay_groups, checkpoint_file="replay_ids.pkl"):
        #Get all replay IDs from groups
        checkpoint_path = self.checkpoint_dir / checkpoint_file

        # Try to load from checkpoint
        if checkpoint_path.exists():
            print(f"Loading replay IDs from checkpoint...")
            with open(checkpoint_path, 'rb') as f:
                return pickle.load(f)

        print(f"Fetching replay IDs from {len(replay_groups)} groups...")
        replay_ids = []

        for i, group_id in enumerate(replay_groups, 1):
            try:
                data = self._make_request(f"{self.base_url}/replays?group={group_id}")
                group_replays = [replay['id'] for replay in data.get('list', [])]
                replay_ids.extend(group_replays)
                print(f"  [{i}/{len(replay_groups)}] Group {group_id}: {len(group_replays)} replays")

            except Exception as e:
                print(f"Error: {e}")
                continue

        # Save checkpoint
        with open(checkpoint_path, 'wb') as f:
            pickle.dump(replay_ids, f)

        print(f"Found {len(replay_ids)} total replays")
        return replay_ids

    def process_replays_streaming(self, replay_ids, batch_size=100, checkpoint_file="processed_data.csv"):
        #Process replays in batches and stream to CSV
        checkpoint_path = self.checkpoint_dir / checkpoint_file

        # Check what's already processed
        processed_ids = set()
        if checkpoint_path.exists():
            existing_df = pd.read_csv(checkpoint_path)
            if 'replay_id' in existing_df.columns:
                processed_ids = set(existing_df['replay_id'].values)
                print(f"Resuming from checkpoint: {len(processed_ids)} already processed")

        remaining_ids = [rid for rid in replay_ids if rid not in processed_ids]
        print(f"Processing {len(remaining_ids)} replays ({len(processed_ids)} already done)")

        self.stats['start_time'] = datetime.now()

        for i, replay_id in enumerate(remaining_ids, 1):
            try:
                data = self._make_request(f"{self.base_url}/replays/{replay_id}")
                row = self._parse_replay(data, replay_id)

                # Append to CSV
                df = pd.DataFrame([row])
                df.to_csv(checkpoint_path, mode='a', header=not checkpoint_path.exists(), index=False)

                self.stats['replays_processed'] += 1

                # Progress update
                if i % 10 == 0:
                    self._print_progress(i, len(remaining_ids))

            except Exception as e:
                print(f"Error processing replay {replay_id}: {e}")
                continue

        print(f"\nProcessing complete!")
        self._print_stats()

        return pd.read_csv(checkpoint_path)

    def _parse_replay(self, data, replay_id):
        # Parse replay JSON into flat dictionary
        row = {'replay_id': replay_id}
        row['created'] = data.get('created', None)

        # Team info
        row['team_name'] = data.get('blue', {}).get('name', 'Unknown')
        row['opp_name'] = data.get('orange', {}).get('name', 'Unknown')

        # Player names - pad to 3 players per team
        blue_players = data.get('blue', {}).get('players', [])
        orange_players = data.get('orange', {}).get('players', [])

        # Pad blue team to 3 players
        for i in range(1, 4):
            if i <= len(blue_players):
                row[f'team{i}'] = blue_players[i-1].get('name', f'Player{i}')
            else:
                row[f'team{i}'] = 'None'

        # Pad orange team to 3 players
        for i in range(1, 4):
            if i <= len(orange_players):
                row[f'opp{i}'] = orange_players[i-1].get('name', f'Player{i}')
            else:
                row[f'opp{i}'] = 'None'

        # Calculate score
        blue_goals = sum(p.get('stats', {}).get('core', {}).get('goals', 0) for p in blue_players)
        orange_goals = sum(p.get('stats', {}).get('core', {}).get('goals', 0) for p in orange_players)

        row['blue_goals'] = blue_goals
        row['orange_goals'] = orange_goals
        row['win'] = blue_goals > orange_goals

        return row

    def _print_progress(self, current, total):
        #Print progress with ETA
        elapsed = (datetime.now() - self.stats['start_time']).total_seconds()
        rate = current / elapsed if elapsed > 0 else 0
        remaining = total - current
        eta_seconds = remaining / rate if rate > 0 else 0

        print(f"  [{current}/{total}] {current/total*100:.1f}% | "
              f"Rate: {rate:.1f} replays/min | "
              f"ETA: {eta_seconds/60:.1f} min")

    def _print_stats(self):
        #Print collection statistics
        elapsed = (datetime.now() - self.stats['start_time']).total_seconds()
        print(f"\nStatistics:")
        print(f"  Total requests: {self.stats['requests_made']}")
        print(f"  Replays processed: {self.stats['replays_processed']}")
        print(f"  Errors: {self.stats['errors']}")
        print(f"  Time elapsed: {elapsed/60:.1f} minutes")
        print(f"  Average rate: {self.stats['replays_processed']/elapsed*60:.1f} replays/min")

In [ ]:
collector = BallchasingCollector(token=Ballchasing_Token, checkpoint_dir='/content/drive/MyDrive/Public RL/Complete ELO Model/checkpoints')
groups = ['2-finals-211ggvb3eb']

In [ ]:
replay_groups = collector.discover_replay_groups(groups)

Loading replay groups from checkpoint...


In [ ]:
replay_ids = collector.get_replay_ids(replay_groups)

Loading replay IDs from checkpoint...


In [ ]:
df = collector.process_replays_streaming(replay_ids)

Resuming from checkpoint: 257 already processed
Processing 0 replays (257 already done)

Processing complete!

Statistics:
  Total requests: 0
  Replays processed: 0
  Errors: 0
  Time elapsed: 0.0 minutes
  Average rate: 0.0 replays/min


##Data Preparation and Model Construction

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Public RL/Complete ELO Model/Legacy Data/Through_2026_Open1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Public RL/Complete ELO Model/Legacy Data/Fifae.csv')
df = pd.concat([df1, df2], ignore_index=True)
df['created'] = pd.to_datetime(df['created'])
df = df.sort_values(by='created', ascending=True)

In [ ]:
#import dataframe and remove content exhibition matches
df_3s = df.dropna()
df_3s = df_3s[((df_3s['team_name'].str.lower() != 'first touch') & (df_3s['opp_name'].str.lower() != 'first_touch'))].reset_index(drop=True)

#condense team/opp values to 2 lists of player names
df_3s['team_players'] = df_3s[['team1','team2','team3']].apply(lambda row: row.to_list(), axis=1)
df_3s['opp_players'] = df_3s[['opp1','opp2','opp3']].apply(lambda row: row.to_list(), axis=1)
#recode win as numerical
df_3s['win'] = df_3s['win'].map({True: 1, False:0})
#sort dataframe for correct weighting later
df_3s = df_3s.sort_values(by='created', ascending=True)
#reduce data to relevant dimensions
data = df_3s.copy()
data = data[['team_players','opp_players','win']].reset_index(drop=True)

data


,team_players,opp_players,win
0,"[LCT, ballerrees, k]","[Dansku_, Porsas52, regser]",0
1,"[LCT, ballerrees, k]","[regser, Dansku_, Porsas52]",0
2,"[ballerrees, LCT, k]","[regser, Porsas52, Dansku_]",0
3,"[regser, Dansku_, Porsas52]","[loyal., Breezi, Kevin]",1
4,"[Dansku_, regser, Porsas52]","[Kevin, loyal., Breezi]",0
...,...,...,...
7463,"[Rw9, Kiileerrz, trk511]","[zen, juicy, vatira]",0
7464,"[trk511, Rw9, Kiileerrz]","[vatira, juicy, zen]",1
7465,"[trk511, Kiileerrz, Rw9]","[vatira, zen, juicy]",0
7466,"[Rw9, Kiileerrz, trk511]","[zen, vatira, juicy]",0


In [ ]:
class RocketLeagueDataset(Dataset):
  def __init__(self, data, apply_recency=True, decay_rate=0.01):
    super().__init__()
    self.team_features = torch.LongTensor(data['team_sequences'].tolist())
    self.opp_features = torch.LongTensor(data['opp_sequences'].tolist())
    self.labels = torch.LongTensor(data['win'])

    if apply_recency:
      n = self.labels.size(0)
      indices = np.arange(n)
      weights = np.exp(decay_rate * indices)

      self.weights = torch.FloatTensor(weights / weights.mean())
    else:
      self.weights = torch.ones(len(self.labels))

  def __len__(self):
    return self.team_features.size(0)

  def __getitem__(self, idx):
    return self.team_features[idx], self.opp_features[idx], self.labels[idx], self.weights[idx]

In [ ]:
aliases ={
    'aboturki': ['abo turki'],
    'acronik' : ['acronik 0', 'acro'],
    'aqua' : ['ahyqua'],
    'ajg' : ['aje','ajf'],
    'applesous' : ['apples' , 'appelsous'],
    'arafar' : ['ara'],
    'archie' : ['arch'],
    'arju' : ['ario'],
    'atomik' : ['ltk_atomik'],
    'atow' : ['atow rikow'],
    'aztromick' : ['aztr'],
    'badnezz' : ['badnezzrl','badnezzski7'],
    'bradk1ng' : ['brad', 'brad la chef', 'bradquentaog7s', 'bradsonmalado', 'lebrad', 'k1ng', 'prime brad'],
    'breezi' : ['breezy'],
    'brenox' : ['brenox3k'],
    'cheese' : ['cheee'],
    'colonel' : ['colonel is lagging'],
    'crr' : ['crn', 'crrdd'],
    'davitrox' : ['davi'],
    'droppz' : ['droppzk3k'],
    'eekso' : ['eeko', 'eeksoszn'],
    'eliakim' : ['eliakimza'],
    'ellil' : ['ell1srai', 'elil'],
    'evoh' : ['evohpaniniwine', 'oevoh'],
    'exfusion_' : ['exfusionz_2'],
    'fiberr' : ['pwr fiberr'],
    'giuk' : ['giuk_rl'],
    'gus' : ['gustexioz'],
    'hyderr' : ['hyderr new binds'],
    'insp1re' : ['inspire'],
    'israkan' : ['israkan 515'],
    'ivan' : ['ivn'],
    'joyo' : ['52 300lbs chud'],
    'jweyts' : ['jweytski7'],
    'kevinacho' : ['kevin', 'kevinacho ama pl'],
    'kaka' : ['kaka0mame'],
    'klaus' : ['klaus queridinho', 'klausrl1'],
    'kofyr' : ['koflii'],
    'kv1' : ['kv1exe'],
    'leodknn' : ['leodkn1'],
    'life' : ['life made day 4'],
    'lucas06' : ['luca06'],
    'lxucha' : ['lxucha lpwr'],
    'lynx' : ['lyn', 'lynx555'],
    'm7md' : ['m7md97', 'm7md97_rl'],
    'majicbear' : ['majibear'],
    'mass' : ['massrl'],
    'mech' : ['mech streamin', 'mechnew setupunrusting', 'nomech'],
    'misery' : ['misery lc'],
    'mtzr' : ['mtzrito', 'mtzr_'],
    'nmj' : ['njm515'],
    'noahsaki' : ['19noahsaki'],
    'nym' : ['nym0'],
    'plgabriel' : ['plgabriel ama reis'],
    'radosin' : ['radosin75', 'radosinho'],
    'reis' : ['reis ama kevinacho', 'reis zoo', 'reisss'],
    'retals' : ['retal'],
    'reysbull' : ['reysmalado', 'reysss'],
    'roods' : ['roodsrl'],
    'rizon' : ['rizun'],
    'rts' : ['rtsz'],
    'saizen' : ['saizen rl', '17saizen'],
    'simas' : ['12simas'],
    'sweaty' : ['sweaty_clarence','sweaty_mclarence'],
    'scrzbbles' : ['scrib', 'scribkep', 'scrib buyssgbundle'],
    'stain' : ['stainy'],
    'teschow' : ['teschow rikow'],
    'twnzr' : ['twnzr13'],
    'valid' : ['valid broken ulna', 'validvalidvalidvvalidvalidvalid', 'gz validraxxus', 'gz validzetsubusoro'],
    'viitin': ['viitin new bind'],
    'vulty' : ['vultacus_ live'],
    'wellace' : ['well1']
}

filename = '/content/drive/MyDrive/Public RL/Complete ELO Model/aliases.pkl'
with open(filename, 'wb') as f:
  pickle.dump(aliases, f)

In [ ]:
#receives: list of player names
#outputs: cleaned list of names
def clean_names_list(names_list):
  for i in range(len(names_list)):
    names_list[i] = re.sub(r'[^\w\s]','',names_list[i].lower()).strip()
  return names_list

#receives: list of cleaned player names
#outputs: aliased series of team lists
def clean_aliases(team_players, aliases):
  full_aliases = {word: key for key in aliases.keys() for word in aliases[key]}
  return [full_aliases[p] if p in full_aliases else p for p in team_players ]

#receives: 2 series of player lists
#outputs: vocab dict for embedding
def build_vocab(team_players, opp_players, aliases):
  vocab = {'<PAD>':0}
  all_tokens = []
  team_players = team_players.apply(clean_names_list)
  opp_players = opp_players.apply(clean_names_list)
  team_players = team_players.apply(clean_aliases, args=(aliases,))
  opp_players = opp_players.apply(clean_aliases, args=(aliases,))
  for team in team_players:
    all_tokens.extend(team)
  for opp in opp_players:
    all_tokens.extend(opp)
  player_counts = Counter(all_tokens)
  for idx, (player, _) in enumerate(player_counts.most_common(), start=1):
    vocab[player] = idx
  return vocab

#testing method
#receives: series of player lists
#outputs: list of unique player names
def check_names(team_players, opp_players):
  all_names = []
  for team in team_players:
    all_names.extend(team)
  for opp in opp_players:
    all_names.extend(opp)
  return sorted(set(all_names))

#receives: list of player names, embedding vocab dict
#outputs: player name sequence
def team_to_sequence(team_players, vocab, aliases):
  return [vocab.get(p, 0) for p in clean_aliases(clean_names_list(team_players), aliases)]

#receives: sequence of player name indices
#outputs: padded sequence
def pad_sequence(sequence, max_len):
  if len(sequence) >= max_len:
    return sequence[:max_len]
  else:
    return sequence + [0] * (max_len - len(sequence))

#receives: 2 series of player name lists, vocab dict, max sequence length int
#outputs: 2 series of padded team sequences
def convert_and_pad_sequences(team_players, opp_players, vocab, aliases, max_len):
  team_sequences = [team_to_sequence(team, vocab, aliases) for team in team_players]
  opp_sequences = [team_to_sequence(opp, vocab, aliases) for opp in opp_players]
  padded_team_sequences = [pad_sequence(seq, max_len) for seq in team_sequences]
  padded_opp_sequences = [pad_sequence(seq, max_len) for seq in opp_sequences]
  return padded_team_sequences, padded_opp_sequences


#receives: 2 series of player name lists, max_sequence length int
#outputs: 2 series of padded player name sequences, vocab dict
def preprocessing_pipeline(team_players, opp_players, aliases, max_len):
  vocab = build_vocab(team_players, opp_players, aliases)
  team_sequences, opp_sequences = convert_and_pad_sequences(team_players, opp_players, vocab, aliases, max_len)
  return team_sequences, opp_sequences, vocab

In [ ]:
def add_reverse_fixtures(frame):
  reverse_frame = frame.copy()
  reverse_frame = reverse_frame[['opp_sequences', 'team_sequences', 'win']]
  reverse_frame['win'] = 1 - reverse_frame['win']
  reverse_frame.columns = ['team_sequences', 'opp_sequences', 'win']

  frame = frame.reset_index()
  reverse_frame = reverse_frame.reset_index()

  frame = frame.rename(columns={'index': 'order'})
  reverse_frame = reverse_frame.rename(columns={'index': 'order'})
  frame['order'] = frame['order'] * 2
  reverse_frame['order'] = reverse_frame['order'] * 2 + 1

  combined = pd.concat([frame, reverse_frame], ignore_index=True)
  combined = combined.sort_values(by='order', ascending=True).drop('order', axis=1).reset_index(drop=True)

  return combined

In [ ]:
#separate train, test, and valid data
max_len = 3

model_data = data.copy()
model_data['team_sequences'], model_data['opp_sequences'], vocab = preprocessing_pipeline(model_data['team_players'], model_data['opp_players'], aliases, max_len)
model_data = model_data[['team_sequences','opp_sequences', 'win']]

#prepare training and test sets

train, test = model_data.loc[:round(model_data.shape[0]*0.9)], model_data[round(model_data.shape[0]*0.9)+1:]
train, valid = train_test_split(train, test_size=0.2, random_state=42)
train = train.sort_index()
valid = valid.sort_index()
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

train = add_reverse_fixtures(train)
valid = add_reverse_fixtures(valid)

test = test.reset_index(drop=True)

In [ ]:
'''
class to implement early stopping

takes: patience (int), delta (float), verbose (bool)

methods: check_early_stop(val_loss) - outputs boolean to stop training
'''
class EarlyStopping():
    def __init__(self, patience=5, delta=0, verbose=False):
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.best_loss = None
        self.no_improvement_count = 0
        self.stop_training = False
    def check_early_stop(self, val_loss):
        if self.best_loss is None or val_loss< self.best_loss-self.delta:
            self.best_loss = val_loss
            self.no_improvement_count = 0
        else:
            self.no_improvement_count += 1
            if self.no_improvement_count >= self.patience:
                self.stop_training = True
                if self.verbose:
                    print('Stopping early due to no/negligible improvment')


In [ ]:
class PoolPredictionNet(nn.Module):
  def __init__(self, vocab_size, embed_dim=64, fc_hidden_dim1=256, fc_hidden_dim2=64, dropout1=0.3, dropout2=0.3, num_heads=4):
    super().__init__()
    self.embedding = nn.Embedding(
        vocab_size,
        embed_dim
    )
    self.attention = nn.MultiheadAttention(embed_dim, num_heads=num_heads, batch_first=True)
    self.fc1 = nn.Linear(embed_dim*6, fc_hidden_dim1)
    self.bn1 = nn.BatchNorm1d(fc_hidden_dim1)
    self.relu1 = nn.ReLU()
    self.dropout1 = nn.Dropout(dropout1)
    self.fc2 = nn.Linear(fc_hidden_dim1, fc_hidden_dim2)
    self.bn2 = nn.BatchNorm1d(fc_hidden_dim2)
    self.relu2 = nn.ReLU()
    self.dropout2 = nn.Dropout(dropout2)
    self.fc3 = nn.Linear(fc_hidden_dim2, 1)
    self.sigmoid = nn.Sigmoid()

    init.kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
    init.kaiming_uniform_(self.fc2.weight, nonlinearity='relu')
    init.xavier_uniform_(self.fc3.weight)

  def forward(self, team_features, opp_features):
    team_embeddings = self.embedding(team_features)
    opp_embeddings = self.embedding(opp_features)

    team_attended, _ = self.attention(team_embeddings, team_embeddings, team_embeddings)
    opp_attended, _ = self.attention(opp_embeddings, opp_embeddings, opp_embeddings)

    team_mean = team_attended.mean(dim=1)        # Average skill/style
    team_max = team_attended.max(dim=1)[0]       # Star player effect
    team_min = team_attended.min(dim=1)[0]       # Weakest link

    opp_mean = opp_attended.mean(dim=1)
    opp_max = opp_attended.max(dim=1)[0]
    opp_min = opp_attended.min(dim=1)[0]


    x = torch.cat([team_mean, team_max, team_min, opp_mean, opp_max, opp_min], dim=1)

    x= self.relu1(self.bn1(self.fc1(x)))
    x = self.dropout1(x)
    x = self.relu2(self.bn2(self.fc2(x)))
    x = self.dropout2(x)
    x = self.sigmoid(self.fc3(x))

    return x

In [ ]:
def train_and_validate(model, train, valid, lr, batch_size, weight_decay=1e-5, recency_decay = 0.01, epochs=100, patience=5, shuffle=True):
  #instantiate dataloaders
  dataset_train = RocketLeagueDataset(train, apply_recency=True, decay_rate=recency_decay)
  dataset_valid = RocketLeagueDataset(valid, apply_recency=True, decay_rate=recency_decay)

  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=shuffle)
  dataloader_valid = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)

  criterion = nn.BCELoss(reduction='none')
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
  early_stopping = EarlyStopping(patience=patience)

  for epoch in range(epochs):
    #training section
    train_loss = 0
    model.train()
    for team_features, opp_features, labels, weights in dataloader_train:
      model.zero_grad()
      outputs = model(team_features, opp_features)

      loss = criterion(outputs, labels.unsqueeze(1).float())
      weighted_loss = (loss * weights.unsqueeze(1)).mean()


      weighted_loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      optimizer.step()

      train_loss += weighted_loss.item()
    train_loss /= len(dataloader_train)

    #validation section
    val_loss = 0
    model.eval()
    for team_features, opp_features, labels, _ in dataloader_valid:
      with torch.no_grad():
        outputs = model(team_features, opp_features)
        loss = criterion(outputs, labels.unsqueeze(1).float()).mean()
        val_loss += loss.item()
    val_loss /= len(dataloader_valid)
    scheduler.step(val_loss)

    early_stopping.check_early_stop(val_loss)
    if early_stopping.stop_training:
      print(f'Early stopping at epoch {epoch}')
      return val_loss, model
  return val_loss, model


In [ ]:
def objective(trial):
  embed_dim = trial.suggest_categorical('embed_dim', [64, 128])
  num_heads = trial.suggest_categorical('num_heads', [2, 4, 8])
  fc_hidden_dim1 = trial.suggest_categorical('fc_hidden_dim1', [256])
  fc_hidden_dim2 = trial.suggest_categorical('fc_hidden_dim2', [32, 64])
  dropout1 = trial.suggest_float('dropout1', 0.1, 0.3, step=0.1)
  dropout2 = trial.suggest_float('dropout2', 0.2, 0.5, step=0.1)
  lr = trial.suggest_float('lr', 0.0001, 0.01, log=True)
  batch_size = trial.suggest_categorical('batch_size', [32])
  weight_decay = trial.suggest_float('weight_decay', 0.0003, 0.001, log=True)
  recency_decay = trial.suggest_float('recency_decay', 1e-5, 3e-5, log=True)

  # Constraint: embed_dim must be divisible by num_heads
  if embed_dim % num_heads != 0:
    raise optuna.exceptions.TrialPruned()

  model = PoolPredictionNet(
      vocab_size=len(vocab),
      embed_dim=embed_dim,
      num_heads=num_heads,
      fc_hidden_dim1=fc_hidden_dim1,
      fc_hidden_dim2=fc_hidden_dim2,
      dropout1=dropout1,
      dropout2=dropout2,
  )

  val_loss, _ = train_and_validate(model, train, valid, lr, batch_size, weight_decay, recency_decay)
  return val_loss

In [ ]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# print('Best hyperparameters', study.best_params)
# print('Best validation loss', study.best_value)

In [ ]:
model = PoolPredictionNet(
    vocab_size=len(vocab),
    embed_dim=64,
    fc_hidden_dim1=256,
    fc_hidden_dim2=64,
    dropout1=0.3,
    dropout2=0.3,

)
_, model = train_and_validate(model, train, valid, lr=0.0020102639361487444, batch_size=32, weight_decay=0.0009504475883440148, recency_decay=1.2616293408204758e-05)

dataset_test = RocketLeagueDataset(test, apply_recency=False)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

acc = Accuracy(task='binary')
model.eval()
with torch.no_grad():
  for team_features, opp_features, labels, _ in dataloader_test:
    test_probs = model(team_features, opp_features)
    test_preds = (test_probs > 0.5)
    acc(test_preds, labels.unsqueeze(1))
accuracy = acc.compute()
print(f'Test Accuracy: {accuracy}')

Early stopping at epoch 12
Test Accuracy: 0.6528149843215942


In [ ]:
train_final, valid_final = train_test_split(model_data, test_size=0.2, random_state=42)
train_final = train_final.sort_index()
valid_final = valid_final.sort_index()
train_final = train_final.reset_index(drop=True)
valid_final = valid_final.reset_index(drop=True)

train_final = add_reverse_fixtures(train_final)
valid_final = add_reverse_fixtures(valid_final)

model = PoolPredictionNet(
    vocab_size=len(vocab),
    embed_dim=64,
    fc_hidden_dim1=256,
    fc_hidden_dim2=64,
    dropout1=0.3,
    dropout2=0.3,

)

_, model = train_and_validate(model, train_final, valid_final, lr=0.010327914968888427, batch_size=64, weight_decay=0.00169633048876229329, recency_decay=1.0042838177194864e-06)

Early stopping at epoch 15


In [ ]:
path = '/content/drive/MyDrive/Public RL/Complete ELO Model/model_state_dict.pth'
torch.save(model.state_dict(), path)

filename = '/content/drive/MyDrive/Public RL/Complete ELO Model/vocab_dict.pkl'
with open(filename, "wb") as file:
    pickle.dump(vocab, file)

<hr><hr><br>

# Prediction Section
<p>Below is the code necessary to make predictions. Run each segment until the next instructions block.</p>
<br><hr><hr>

In [ ]:
%pip install torchmetrics
%pip install optuna
%pip install gdown

import time
import requests
import re
import random
import pickle
import json
import torch
import math
import random
import optuna
import gdown

import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn.functional as f
import torch.nn.init as init

from collections import deque, Counter
from datetime import datetime, timedelta
from scipy import stats
from torch import nn
from torchmetrics import Accuracy
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from pathlib import Path
from google.colab import userdata
from google.colab import files
from google.colab import drive

In [ ]:
class PoolPredictionNet(nn.Module):
  def __init__(self, vocab_size, embed_dim=64, fc_hidden_dim1=256, fc_hidden_dim2=64, dropout1=0.3, dropout2=0.3, num_heads=4):
    super().__init__()
    self.embedding = nn.Embedding(
        vocab_size,
        embed_dim
    )
    self.attention = nn.MultiheadAttention(embed_dim, num_heads=num_heads, batch_first=True)
    self.fc1 = nn.Linear(embed_dim*6, fc_hidden_dim1)
    self.bn1 = nn.BatchNorm1d(fc_hidden_dim1)
    self.relu1 = nn.ReLU()
    self.dropout1 = nn.Dropout(dropout1)
    self.fc2 = nn.Linear(fc_hidden_dim1, fc_hidden_dim2)
    self.bn2 = nn.BatchNorm1d(fc_hidden_dim2)
    self.relu2 = nn.ReLU()
    self.dropout2 = nn.Dropout(dropout2)
    self.fc3 = nn.Linear(fc_hidden_dim2, 1)
    self.sigmoid = nn.Sigmoid()

    init.kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
    init.kaiming_uniform_(self.fc2.weight, nonlinearity='relu')
    init.xavier_uniform_(self.fc3.weight)

  def forward(self, team_features, opp_features):
    team_embeddings = self.embedding(team_features)
    opp_embeddings = self.embedding(opp_features)

    team_attended, _ = self.attention(team_embeddings, team_embeddings, team_embeddings)
    opp_attended, _ = self.attention(opp_embeddings, opp_embeddings, opp_embeddings)

    team_mean = team_attended.mean(dim=1)        # Average skill/style
    team_max = team_attended.max(dim=1)[0]       # Star player effect
    team_min = team_attended.min(dim=1)[0]       # Weakest link

    opp_mean = opp_attended.mean(dim=1)
    opp_max = opp_attended.max(dim=1)[0]
    opp_min = opp_attended.min(dim=1)[0]


    x = torch.cat([team_mean, team_max, team_min, opp_mean, opp_max, opp_min], dim=1)

    x= self.relu1(self.bn1(self.fc1(x)))
    x = self.dropout1(x)
    x = self.relu2(self.bn2(self.fc2(x)))
    x = self.dropout2(x)
    x = self.sigmoid(self.fc3(x))

    return x


  #receives: list of player names
#outputs: cleaned list of names
def clean_names_list(names_list):
  for i in range(len(names_list)):
    names_list[i] = re.sub(r'[^\w\s]','',names_list[i].lower()).strip()
  return names_list

#receives: list of cleaned player names
#outputs: aliased series of team lists
def clean_aliases(team_players, aliases):
  full_aliases = {word: key for key in aliases.keys() for word in aliases[key]}
  return [full_aliases[p] if p in full_aliases else p for p in team_players ]

#receives: 2 series of player lists
#outputs: vocab dict for embedding
def build_vocab(team_players, opp_players, aliases):
  vocab = {'<PAD>':0}
  all_tokens = []
  team_players = team_players.apply(clean_names_list)
  opp_players = opp_players.apply(clean_names_list)
  team_players = team_players.apply(clean_aliases, args=(aliases,))
  opp_players = opp_players.apply(clean_aliases, args=(aliases,))
  for team in team_players:
    all_tokens.extend(team)
  for opp in opp_players:
    all_tokens.extend(opp)
  player_counts = Counter(all_tokens)
  for idx, (player, _) in enumerate(player_counts.most_common(), start=1):
    vocab[player] = idx
  return vocab

#testing method
#receives: series of player lists
#outputs: list of unique player names
def check_names(team_players, opp_players):
  all_names = []
  for team in team_players:
    all_names.extend(team)
  for opp in opp_players:
    all_names.extend(opp)
  return sorted(set(all_names))

#receives: list of player names, embedding vocab dict
#outputs: player name sequence
def team_to_sequence(team_players, vocab, aliases):
  return [vocab.get(p, 0) for p in clean_aliases(clean_names_list(team_players), aliases)]

#receives: sequence of player name indices
#outputs: padded sequence
def pad_sequence(sequence, max_len):
  if len(sequence) >= max_len:
    return sequence[:max_len]
  else:
    return sequence + [0] * (max_len - len(sequence))

#receives: 2 series of player name lists, vocab dict, max sequence length int
#outputs: 2 series of padded team sequences
def convert_and_pad_sequences(team_players, opp_players, vocab, aliases, max_len):
  team_sequences = [team_to_sequence(team, vocab, aliases) for team in team_players]
  opp_sequences = [team_to_sequence(opp, vocab, aliases) for opp in opp_players]
  padded_team_sequences = [pad_sequence(seq, max_len) for seq in team_sequences]
  padded_opp_sequences = [pad_sequence(seq, max_len) for seq in opp_sequences]
  return padded_team_sequences, padded_opp_sequences


#receives: 2 series of player name lists, max_sequence length int
#outputs: 2 series of padded player name sequences, vocab dict
def preprocessing_pipeline(team_players, opp_players, aliases, max_len):
  vocab = build_vocab(team_players, opp_players, aliases)
  team_sequences, opp_sequences = convert_and_pad_sequences(team_players, opp_players, vocab, aliases, max_len)
  return team_sequences, opp_sequences, vocab



class RocketLeagueDataset(Dataset):
  def __init__(self, data, apply_recency=True, decay_rate=0.01):
    super().__init__()
    self.team_features = torch.LongTensor(data['team_sequences'].tolist())
    self.opp_features = torch.LongTensor(data['opp_sequences'].tolist())
    self.labels = torch.LongTensor(data['win'])

    if apply_recency:
      n = self.labels.size(0)
      indices = np.arange(n)
      weights = np.exp(decay_rate * indices)

      self.weights = torch.FloatTensor(weights / weights.mean())
    else:
      self.weights = torch.ones(len(self.labels))

  def __len__(self):
    return self.team_features.size(0)

  def __getitem__(self, idx):
    return self.team_features[idx], self.opp_features[idx], self.labels[idx], self.weights[idx]


In [ ]:
MODEL_FILE_ID = "1zbwMlsNe5MKJsZV4AHTTZ_wZHDMleziI"
VOCAB_FILE_ID = "1e0ijSq9TVTdfm36bqR8925u7dqw7WeFv"
ALIASES_FILE_ID = "1ou5x9zX8kU8y_hLFgWNsOmxe7AxxbAjF"

# Download model state dict
print("Downloading model state dict...")
model_path = "model_state_dict.pth"
gdown.download(f"https://drive.google.com/uc?id={MODEL_FILE_ID}", model_path, quiet=False, fuzzy=True)

# Download vocab dictionary
print("Downloading vocab dictionary...")
vocab_path = "vocab_dict.pkl"
gdown.download(f"https://drive.google.com/uc?id={VOCAB_FILE_ID}", vocab_path, quiet=False, fuzzy=True)

# Download aliases
print("Downloading aliases...")
aliases_path = "aliases.pkl"
gdown.download(f"https://drive.google.com/uc?id={ALIASES_FILE_ID}", aliases_path, quiet=False, fuzzy=True)

# Load the files
print("Loading model state dict...")
model_state_dict = torch.load(model_path, map_location='cpu')

print("Loading vocab dictionary...")
with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)

print("Loading aliases...")
with open(aliases_path, 'rb') as f:
    aliases = pickle.load(f)

print("Download and loading complete!")
print(f"Model keys: {list(model_state_dict.keys())[:5]}...")
print(f"Vocab size: {len(vocab)}")
print(f"Aliases loaded: {len(aliases) if isinstance(aliases, (dict, list)) else 'N/A'}")


Downloading...
From: https://drive.google.com/uc?id=1zbwMlsNe5MKJsZV4AHTTZ_wZHDMleziI
To: /content/model_state_dict.pth
100%|██████████| 729k/729k [00:00<00:00, 27.5MB/s]


Downloading...
From: https://drive.google.com/uc?id=1e0ijSq9TVTdfm36bqR8925u7dqw7WeFv
To: /content/vocab_dict.pkl
100%|██████████| 8.77k/8.77k [00:00<00:00, 17.8MB/s]


Downloading...
From: https://drive.google.com/uc?id=1ou5x9zX8kU8y_hLFgWNsOmxe7AxxbAjF
To: /content/aliases.pkl
100%|██████████| 2.13k/2.13k [00:00<00:00, 7.51MB/s]

Loading model state dict...
Loading vocab dictionary...
Loading aliases...
Download and loading complete!
Model keys: ['embedding.weight', 'attention.in_proj_weight', 'attention.in_proj_bias', 'attention.out_proj.weight', 'attention.out_proj.bias']...
Vocab size: 739
Aliases loaded: 72


In [ ]:
#instantiate model with downloaded structure and vocab

model = PoolPredictionNet(len(vocab))
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [ ]:
def simulate_series(pred_team, pred_opp, vocab, aliases, max_len=3, series_length=5, sim_length=1000):
  pred_team = pad_sequence(team_to_sequence(pred_team, vocab, aliases), max_len)
  pred_opp = pad_sequence(team_to_sequence(pred_opp, vocab, aliases), max_len)

  pred_team = torch.LongTensor(pred_team).unsqueeze(0)
  pred_opp = torch.LongTensor(pred_opp).unsqueeze(0)

  model.eval()
  with torch.no_grad():
    team_prob_first = float(model(pred_team, pred_opp).squeeze())
    team_prob_second = 1-float(model(pred_opp, pred_team).squeeze())

  team_prob = (team_prob_first + team_prob_second) / 2
  team_wins, opp_wins = pd.Series(), pd.Series()
  for i in range(sim_length):
    sim_wins = 0
    sim_losses = 0
    while ((sim_wins < series_length/2) & (sim_losses < series_length/2)):
      if (test := random.random()) < team_prob:
        sim_wins += 1
      else:
        sim_losses += 1
    team_wins[i] = sim_wins
    opp_wins[i] = sim_losses
  team_wins = team_wins.mean()
  opp_wins = opp_wins.mean()

  if team_wins > opp_wins:
    team_wins = math.ceil(series_length/2)
    opp_wins = round(opp_wins, 0)
  else:
    opp_wins = math.ceil(series_length/2)
    team_wins = round(team_wins,0)

  return [int(team_wins), int(opp_wins)], team_prob

<hr><hr><br><p>In the segment below, there is a list of RLCS teams. The team name is on the left, and the list of players is on the right. To simulate a series from these teams, run the cell below and make sure to select teams from the list as they are spelled.</p>
<p>To select any 6 players to simulate a hypothetical matchup, use the next cell to enter players individually. Make sure player names are spelled the same way they are in the vocab list that appears if you run the last cell</p><br><hr><hr>

In [ ]:
teams_dict = {
    'vitality' : ['zen, exotiik','stizzy'],
    'kc' : ['vatira', 'atow', 'juicy'],
    'nip' : ['joreuz', 'crr', 'oaly'],
    'man city' : ['ejby', 'accro', 'tempoh'],
    'geekay' : ['apparentlyjack', 'joyo', 'seikoo'],
    'novo' : ['nico','acronik', 'giuk'],
    'pld' : ['gawfs', 'ethan', 'pluvo'],
    'dr' : ['vorce', 'ne0n', 'motion15'],
    'cloud' : ['jweyts', 'badnezz', 'wozyen'],
    'ght' : ['tms, hyderr', 'gramma'],
    'hogan mode' : ['growlii', 'rehzzy', 'rxii'],
    'gentlebench' : ['yujin', 'radosin', 'mtzr'],
    'tks' : ['thyyder', 'scream', 'kerian'],
    'gentle mates' : ['archie', 'nass', 'oski'],
    'sonics' : ['toxiic', 'mikeboy', 'smokez'],
    'magnifico' : ['atomik', 'tox', 'rezears'],
    'ssg' : ['reveal', 'chronic', 'diaz'],
    'geng' : ['majicbear', 'justin', 'rise'],
    'nrg' : ['atomic', 'daniel', 'beastmode'],
    '100x35' : ['crispy', 'simas', 'pzy'],
    'lotus' : ['xprt', 'noly', 'wellace'],
    'redacted' : ['wahvey', '2piece', 'tawk'],
    'sk' : ['arju', 'relatingwave', 'speed'],
    'rebellion' : ['firstkiller', 'kofyr', 'lj'],
    'gas' : ['garrettg', 'ayyjayy', 'squishy'],
    'm80' : ['aris', 'deevo', 'mech'],
    'top leh' : ['comm', 'creamz', 'paarth'],
    'silence' : ['resonal', 'pigeon', 'lev'],
    'unreal nightmare' : ['druee', 'gyro', 'life'],
    'feastabonium' : ['s5cosmic', 'ahduhm', 'hazo'],
    'fut' : ['cheese', 'frosty', 'sosa'],
    'ciel' : ['pndh', 'percy', 'night'],
    'pwr' : ['fiberr', 'gus', 'superlachie'],
    'wildcard' : ['fever', 'torsos', 'bananahead'],
    'furia' : ['yanxnz', 'lostt', 'swiftt'],
    'mibr' : ['reysbull', 'aztromick', 'sad'],
    'falcons' : ['rw9', 'kiileerrz', 'dralii'],
    'twisted minds' : ['nwpo', 'm0nkey m00n', 'trk511'],
    'saudi arabia' : ['rw9', 'kiileerrz', 'trk511'],
    'oman' : ['abdullah', 'lumber', 'sultan'],
    'usa' : ['atomic', 'beastmode', 'daniel'],
    'brazil' : ['yanxnz', 'swiftt', 'lostt'],
    'chile' : ['reysbull', 'davitrox', 'pan'],
    'germany' : ['tox', 'catalysm', 'rezears'],
    'italy' : ['arju', 'davoof', 'hyderr'],
    'norway' : ['wozyen', 'jup', 'bruhkay'],
    'morocco' : ['dralii', 'nass', 'skyrix'],
    'australia' : ['fever', 'torsos', 'bananahead'],
    'france' : ['zen', 'vatira', 'exotiik'],
    'malaysia' : ['sphinx', 'blue', 'misty'],
    'england' : ['apparentlyjack', 'archie', 'joyo'],
    'belgium' : ['aztral', 'atow', 'rysfox'],
    'south africa' : ['2die4', 'gunz', 'snowyy'],
    'netherlands' : ['joreuz', 'oaly', 'mikeboy']
}

In [ ]:
a
a
a
##simulate from teams
team = input('Team 1: ')
opp = input('Team 2: ')
length = int(input('Series Length (odd only): '))

series_score, team_prob = simulate_series(teams_dict[team], teams_dict[opp], vocab, aliases, max_len=3, series_length=length)

print("\n")
print('-'*50)
print(f'{team.title()} {series_score[0]} - {series_score[1]} {opp.title()}')
print(f'{team.title()} Single Game Win Probability: {round(team_prob*100, 2)}%')
print('-'*50)

Team 1: fut
Team 2: nrg
Series Length (odd only): 7


--------------------------------------------------
Fut 2 - 4 Nrg
Fut Single Game Win Probability: 36.85%
--------------------------------------------------


In [ ]:
#simulate series from players (vocab list below)
#if you spell a player name incorrectly, they will be entered as a "default" player
#default players will affect the prediction differently than the correctly spelled player
team1 = input('Team 1, Player 1: ')
team2 = input('Team 1, Player 2: ')
team3 = input('Team 1, Player 3: ')

opp1 = input('Team 2, Player 1: ')
opp2 = input('Team 2, Player 2: ')
opp3 = input('Team 2, Player 3: ')

length = int(input('Series Length (odd only): '))

team = [team1, team2, team3]
opp = [opp1, opp2, opp3]

print('\n')
print(simulate_series(team, opp, vocab, aliases, series_length=length))

Team 1, Player 1: a
Team 1, Player 2: a
Team 1, Player 3: a
Team 2, Player 1: a
Team 2, Player 2: a
Team 2, Player 3: a
Series Length (odd only): 7


([4, 3], 0.5)


In [ ]:
sorted(list(vocab.keys()))

['',
 '21',
 '2die4',
 '2piece',
 '4ever milo',
 '7kexii',
 '7mani',
 '<PAD>',
 'a7md',
 'aan1',
 'abdullah',
 'abo mzoon',
 'abood',
 'aboturki',
 'abscrazy',
 'accro',
 'acerolaas',
 'acronik',
 'aemond targaryen',
 'aguz',
 'ahduhm',
 'ajg',
 'akai',
 'akame',
 'akira0902',
 'alameri',
 'aloneeyy',
 'alphinhaa',
 'alraz',
 'alxx',
 'amatel',
 'amphis',
 'ams',
 'andy',
 'anyeelo',
 'apparentlyjack',
 'applesous',
 'aqua',
 'arafar',
 'archie',
 'aris',
 'arju',
 'arrow',
 'asn_rubiix',
 'atomic',
 'atomik',
 'atow',
 'awareant9767',
 'awkwavey',
 'axrxs',
 'ayman',
 'ayyjayy',
 'aziz',
 'azooz',
 'aztral',
 'aztromick',
 'b',
 'b2sel',
 'baconhero',
 'bad',
 'bader',
 'badnezz',
 'bal',
 'balakeplease2 5',
 'ballerrees',
 'bananacat',
 'bananahead',
 'bathy',
 'baz',
 'bazlenks',
 'beastmode',
 'beasty',
 'bebbangboy',
 'bemmz',
 'ben',
 'benji',
 'besogoat',
 'bfg bossk',
 'bfg dreameh',
 'bfg milo',
 'big gez',
 'bigfoot0',
 'billy',
 'blade',
 'blue',
 'bluii',
 'bnj66',
 'bob',
